# --> NEO4J EXPERIMENTING <--

#### Neo4j Setup Initialization

In [ ]:
from pathlib import Path
import os
from neo4j import GraphDatabase, RoutingControl
from dotenv import load_dotenv

load_dotenv()

class Neo4jManager:
    def __init__(self):
        uri = os.getenv("NEO4J_URI")
        admin_auth = (os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))
        reader_auth = (os.getenv("LLM_READER_USER"), os.getenv("LLM_READER_PASSWORD"))

        # 1. Driver ADMIN (pentru scrieri/upload)
        self._driver_admin = GraphDatabase.driver(uri, auth=admin_auth)

        # 2. Driver READER-ONLY USER (pentru AI Queries)
        self._driver_reader = GraphDatabase.driver(uri, auth=reader_auth)

    def close(self):
        self._driver_admin.close()
        self._driver_reader.close()

    # --- Funcția pe care o folosește Agentul tău ---
    def run_safe_query(self, cypher_query):
        """
        Aceasta este funcția pe care o apelezi din LangGraph.
        Folosește DOAR driverul READER.
        """
        try:
            # Folosim execute_query (metoda modernă din Neo4j 5.x)
            records, _, _ = self._driver_reader.execute_query(
                cypher_query, 
                routing_=RoutingControl.READ
            )
            # Transformăm în JSON simplu pentru LLM
            return [r.data() for r in records]
            
        except Exception as e:
            # Aici prinzi eroarea dacă AI-ul a încercat să șteargă
            if "Forbidden" in str(e):
                return "SECURITY_BLOCK: AI attempted a write operation."
            return f"ERROR: {str(e)}"

    # --- Funcția pentru tine (Backend/Upload) ---
    def run_admin_write(self, cypher_query, params=None):
        """
        Folosită când încarci PDF-uri sau modifici date manual.
        """
        self._driver_admin.execute_query(cypher_query, parameters_=params)



## Azure Configuration

In [10]:
from langchain_openai import AzureChatOpenAI
import os

# Set your Azure credentials
os.environ['AZURE_OPENAI_ENDPOINT'] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ['AZURE_OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ['OPENAI_API_VERSION'] = os.getenv("OPENAI_API_VERSION")
# Create the LLM
llm = AzureChatOpenAI(
            azure_deployment='gpt-4.1-mini',
            model='gpt-4.1-mini',
            temperature=0.0,
        )

# --- Embeddings ---
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding"
)

def get_embeddings(text):
    return embeddings.embed_query(text)


## I/O Functions

In [ ]:
import json

def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            medicaments_data = json.load(f)
    except FileNotFoundError:
        print(f"Eroare: Fișierul de intrare '{file_path}' nu a fost găsit.")
    except json.JSONDecodeError:
        print(f"Eroare: Fișierul '{file_path}' nu conține un JSON valid.")

    return medicaments_data

def save_json_file(file_path, data):
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Datele au fost salvate cu succes in {file_path}")
    except Exception as e:
        print(f"Eroare la salvarea datelor in {file_path}: {e}")
    
medicamente = read_json_file("../data/medic_test.json")
nutrienti = read_json_file("../data/nutrients_test.json")

## Constraints & Indexes

In [ ]:
def setup_graph_database(db):
    db._driver_admin.execute_query("""
        CREATE CONSTRAINT medicament_unique IF NOT EXISTS 
        FOR (m:Medicament) 
        REQUIRE m.name IS UNIQUE
    """)
    db._driver_admin.execute_query("""
        CREATE CONSTRAINT nutrient_unique IF NOT EXISTS 
        FOR (n:Nutrient) 
        REQUIRE n.name IS UNIQUE
    """)
    db._driver_admin.execute_query("""
        CREATE FULLTEXT INDEX medicament_full_search IF NOT EXISTS
        FOR (m:Medicament)
        ON EACH [m.name, m.pharmacologic_class, m.brand_names]
        """)

    db._driver_admin.execute_query("""
        CREATE FULLTEXT INDEX nutrient_full_search IF NOT EXISTS
        FOR (n:Nutrient)
        ON EACH [n.name, n.synonyms]
        """)
        
db = Neo4jManager()
setup_graph_database(db)


## ---> Medicamment NODE <---

In [ ]:
def standardize_name(name):
    if not name:
        return "Unknown"
    
    return name.strip().title()


def create_medicaments_nodes(db,medicament_data):  
    if not db:
        print("Nu există conexiune la baza de date.")
        return
    
    for medicament in medicament_data:
        med_name = standardize_name(medicament.get('medicament'))
        all_brand_names = medicament.get('us_brand_names',[]) + medicament.get('canadian_brand_names',[])

        create_query = """
        MERGE (m:Medicament {
        name: $name,
        brand_names: $brand_names,
        synonyms: $synonyms,
        pharmacologic_class: $pharmacologic_class
        })
        RETURN m.name
        """

        params = {
        "name": med_name,
        "brand_names": all_brand_names,
        "synonyms": medicament.get('synonyms',[]),
        "pharmacologic_class": medicament.get('pharmacologic_class',[])
        }    

        try:
            db._driver_admin.execute_query(create_query,params)
            print(f"Medicamentul {medicament['medicament']} a fost adaugat in KG cu id: {med_name}")
        except Exception as e:
            print(f"Eroare la crearea medicamentului {medicament['medicament']}: {e}")
            continue

create_medicaments_nodes(db,medicamente)

## ---> Pharmacological Class NODE <---

In [ ]:
def create_pharmacologic_class_nodes(db,medicament_data):  
    if not db:
        print("Nu există conexiune la baza de date.")
        return
    
    for medicament in medicament_data:
        med_name = standardize_name(medicament.get('medicament'))
        pharmacologic_class = medicament.get('pharmacologic_class',[])

        for pharma_class in pharmacologic_class:
            create_query = """
            MATCH (m:Medicament {name: $name})

            MERGE (pc:PharmacologicClass {
            pharmacologic_class: $pharmacologic_class})
            ON CREATE SET 
                pc.pharmacologic_class = $pharmacologic_class
            ON MATCH SET 
                pc.pharmacologic_class = $pharmacologic_class

            MERGE (m)-[:Belongs_To]->(pc)

            RETURN pc.name
            """

            params = {
            "name": med_name,
            "pharmacologic_class": pharma_class
            }    

            try:
                db._driver_admin.execute_query(create_query,params)
                print(f"Clasa Pharmacologica {pharma_class} a fost adaugata in KG pentru medicamentul {medicament['medicament']}")
            except Exception as e:
                print(f"Eroare la crearea Clasei Pharmacologice {pharma_class} pentru medicamentul {medicament['medicament']}: {e}")
                continue

create_pharmacologic_class_nodes(db,medicamente)

## ---> Nutrient NODE <---

In [ ]:
def create_nutrients_nodes(db,nutrient_data):  
    if not db:
        print("Nu există conexiune la baza de date.")
        return
    
    for nutrient in nutrient_data:
        nut_name = standardize_name(nutrient.get('nutrient'))

        effects_of_depletion = nutrient.get('effects_of_depletion', '')
        overview = nutrient.get('overview', '')
        biological_function_effect = nutrient.get('biological_function_and_effect', '')
        side_effects_toxicity = nutrient.get('side_effects_and_toxicity', '')
        dosage_range = nutrient.get('dosage_range', '')
        dietary_sources = nutrient.get('dietary_sources', '')
        forms = nutrient.get('forms', '')
        rda = nutrient.get('rda', '')

        text_to_embed = f"{nut_name} {overview} {biological_function_effect}"
        if len(text_to_embed) > 7000:
            print(f"WARNING: NUTRIENTUL {nut_name} pentru embedding este prea lung. Trunchiat la 1536 caractere.")
            print(f"Textul original: {len(text_to_embed)} caractere")
            text_to_embed = text_to_embed[:7000]

        vector_embedding = get_embeddings(text_to_embed)

        if not vector_embedding:
            print(f"WARNING: Nu s-a putut genera embedding pentru {nut_name}")
            continue

        create_query = """
        MERGE (n:Nutrient {name: $name})
        ON CREATE SET 
            n.name = $name,
            n.overview = $overview,
            n.biological_function_effect = $biological_function_effect,
            n.dosage_range = $dosage_range,
            n.biological_function_embedding = $biological_function_embedding,
            n.forms = $forms,
            n.rda = $rda
        ON MATCH SET 
            n.name = $name,
            n.overview = $overview,
            n.biological_function_effect = $biological_function_effect,
            n.dosage_range = $dosage_range,
            n.biological_function_embedding = $biological_function_embedding,
            n.forms = $forms,
            n.rda = $rda
        RETURN n.name
        """

        params = {
        "name": nut_name,
        "overview": overview,
        "biological_function_effect": biological_function_effect,
        "dosage_range": dosage_range,
        "biological_function_embedding": vector_embedding,
        "forms": forms,
        "rda": rda
        }    

        try:
            db._driver_admin.execute_query(create_query,params)
            print(f"Nutrientul {nutrient['nutrient']} a fost adaugat in KG cu id: {nut_name}")
        except Exception as e:
            print(f"Eroare la crearea nutrientului {nutrient['nutrient']}: {e}")
            return None
        
        create_food_source_query = """
        MATCH (n:Nutrient {name: $name})

        MERGE(fs:FoodSource {dietary_source: $food_source})
        ON CREATE SET 
            fs.dietary_source = $food_source
        ON MATCH SET 
            fs.dietary_source = $food_source
        MERGE (n)-[:Found_In]->(fs)
        RETURN n.name, fs.dietary_source
        """

        params_food_source = {
        "name": nut_name,
        "food_source": dietary_sources
        }
        try:
            db._driver_admin.execute_query(create_food_source_query,params_food_source)
            print(f"Relatia dintre nutrientul {nut_name} si sursa alimentara {dietary_sources} a fost adaugata in KG")
        except Exception as e:
            print(f"Eroare la crearea relatiei dintre nutrientul {nut_name} si sursa alimentara {dietary_sources}: {e}")


        create_side_effects_query = """
        MATCH (n:Nutrient {name: $name})

        MERGE(se:SideEffect {side_effect: $side_effect})
        ON CREATE SET 
            se.side_effect = $side_effect
        ON MATCH SET 
            se.side_effect = $side_effect
        MERGE (n)-[:Has_Side_Effect]->(se)
        RETURN n.name, se.side_effect
        """

        params_side_effects = {
        "name": nut_name,
        "side_effect": side_effects_toxicity
        }
        try:
            db._driver_admin.execute_query(create_side_effects_query,params_side_effects)
            print(f"Relatia dintre nutrientul {nut_name} si efectul secundar {side_effects_toxicity} a fost adaugata in KG")
        except Exception as e:
            print(f"Eroare la crearea relatiei dintre nutrientul {nut_name} si efectul secundar {side_effects_toxicity}: {e}")

        print('-'*100)
        
create_nutrients_nodes(db,nutrienti)

## ---> Symptoms NODE <---

In [ ]:

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel

class ExtractedSymptoms(BaseModel):
    canonical_name: str
    layman_variants: list[str]

class ExtractedSymptomsResult(BaseModel):
    symptoms: list[ExtractedSymptoms]

def extract_symptoms(text:str,nutrient:str)->list[ExtractedSymptoms]:
    print(f"Extragem simptomele pentru {nutrient}") 
    SYMPTOM_EXTRACTION_PROMPT = ChatPromptTemplate.from_messages([
        ("system", """You are a clinical symptom extractor. Given a text describing 
    nutrient depletion effects, extract each discrete symptom as a separate entity.

    For each symptom, provide:
    1. The canonical medical term
    2. Common layman/patient descriptions (how a patient might describe it)

    Be exhaustive. Extract ALL symptoms mentioned, even if implicitly stated.
    Output as valid JSON matching the schema."""),
        
        ("user", """Nutrient: {nutrient_name}

    Effects of Depletion Text:
    {effects_text}

    Extract all symptoms:""")
    ])

    chain = SYMPTOM_EXTRACTION_PROMPT | llm.with_structured_output(ExtractedSymptomsResult)
    # print(chain)

    response = chain.invoke({"nutrient_name": nutrient, "effects_text": text})
    print(response)
    return response.symptoms

def create_symptoms_nodes(db,medicament_name,nutrient_name,effects_text,depletion_event_id):  
    if not db:
        print("Nu există conexiune la baza de date.")
        return

    symptoms = extract_symptoms(effects_text,nutrient_name)
    for symptom in symptoms:
        canonical_name = symptom.canonical_name
        layman_variants = symptom.layman_variants

        variants_str = ", ".join(layman_variants) if layman_variants else ""
        text_to_embed = f"{canonical_name} ({variants_str})"

        try:
            vector_embedding = get_embeddings(text_to_embed)
        except Exception as e:
            print(f"Eroare la generarea embedding-ului pentru {text_to_embed}: {e}")
            continue

        create_query = """
        MATCH (e:DepletionEvent {depletion_event_id: $event_uid})

        MERGE (s:Symptom {
        name: $name})
        ON CREATE SET 
            s.layman_variants = $layman_variants,
            s.embedding = $embedding_vector
        ON MATCH SET 
            s.layman_variants = $layman_variants,
            s.embedding = $embedding_vector

        MERGE (e)-[:Has_Symptom]->(s)

        RETURN s.name
        """

        params = {
        "event_uid": depletion_event_id,
        "name": canonical_name,
        "layman_variants": layman_variants,
        "embedding_vector": vector_embedding
        }    

        try:
            db._driver_admin.execute_query(create_query,params)
            print(f"Simptomul {canonical_name} a fost adaugat in KG cu numele {canonical_name}")
        except Exception as e:
            print(f"Eroare la crearea simptomului {canonical_name}: {e}")

## ---> Interaction Event NODE <---

In [ ]:
def create_depletion_event_nodes(db,depletion_event_id):

    create_query = """
    MERGE (depletion_event:DepletionEvent {
    depletion_event_id: $depletion_event_id
    })
    RETURN depletion_event
    """

    params = {
    "depletion_event_id": depletion_event_id,
    }    

    try:
        db._driver_admin.execute_query(create_query,params)
        print(f"Depletion event {depletion_event_id} a fost adaugat in KG")
    except Exception as e:
        print(f"Eroare la crearea depletion event {depletion_event_id}: {e}")
        # continue

def create_depletion_interaction_nodes(db,medicament_data):  
    if not db:
        print("Nu există conexiune la baza de date.")
        return
    
    for medicament in medicament_data:
        med_name = standardize_name(medicament.get('medicament'))
        studies = medicament.get('studies',[])
        for nutrient in medicament.get('nutrients_depleted',[]):
            found = False
            contor_studies = 0
            nutrient_name = standardize_name(nutrient.get('nutrient'))
            effects_text = nutrient.get('potential_risks')
            depletion_event_id = f"DE_{med_name}_{nutrient_name}"
            clean_id = depletion_event_id.replace(", ", "_").replace(",", "_").replace(" ", "_").replace("/", "_")
            print(f"Depletion event ID: {clean_id}")
            create_depletion_event_nodes(db, clean_id)
            create_symptoms_nodes(db,medicament,nutrient_name,effects_text,clean_id)
            create_drug_causes_relation(db,med_name,clean_id)
            create_depletes_relation(db,clean_id,nutrient_name)
            for study in studies:
                contor_studies += 1
                if nutrient_name in study.get('study_title'):
                    print(f"Medicamentul {med_name} si nutrientul {nutrient_name} au fost gasite in studiul {study.get('study_title')}")
                    found = True
                    create_has_evidence_relation(db,study,clean_id)
                if not found:
                    print(f"Medicamentul {med_name} si nutrientul {nutrient_name} nu au fost gasite in studiul {study.get('study_title')}")
        print('-'*100)

create_depletion_interaction_nodes(db,medicamente)

## ---> Relations <---

In [ ]:
def create_drug_causes_relation(db,med_name,depletion_event_id):
    if not db:
        print("Nu există conexiune la baza de date.")
        return
    create_query = """
                    MATCH (m:Medicament {name: $med_name})
                    MATCH (de:DepletionEvent {depletion_event_id: $depletion_event_id})
                    MERGE (m)-[r:CAUSES]->(de)
                    RETURN m.name, de.depletion_event_id, r
                    """
    params = {
        "med_name": med_name,
        "depletion_event_id": depletion_event_id
    }
    try:
        db._driver_admin.execute_query(create_query,params)
        print(f"Relatia {med_name} :CAUSES {depletion_event_id} a fost adaugata in KG")
    except Exception as e:
        print(f"Eroare la crearea relatiei {med_name} :CAUSES {depletion_event_id}: {e}")


def create_depletes_relation(db,clean_id,nutrient_name):
    if not db:
        print("Nu există conexiune la baza de date.")
        return
    
    create_query = """
                    MATCH (de:DepletionEvent {depletion_event_id: $depletion_event_id})
                    MATCH (n:Nutrient {name: $nut_name})
                    MERGE (de)-[r:DEPLETES]->(n)
                    RETURN de.depletion_event_id, n.name, r
                    """
    
    params = {
        "depletion_event_id": clean_id,
        "nut_name": nutrient_name
    }
    try:
        db._driver_admin.execute_query(create_query,params)
        print(f"Relatia {clean_id} :DEPLETES {nutrient_name} a fost adaugata in KG")
    except Exception as e:
        print(f"Eroare la crearea relatiei {clean_id} :DEPLETES {nutrient_name}: {e}")

    
def create_has_evidence_relation(db,study_obj,depl_event_id):
    
    create_query = """
    MATCH (e:DepletionEvent {depletion_event_id: $event_uid})

    MERGE (st:Study {
    study_no: $study_no})
    ON CREATE SET 
        st.study_title = $study_title,
        st.content = $content,
        st.source = $source
    ON MATCH SET 
        st.study_title = $study_title,
        st.content = $content,
        st.source = $source

    MERGE (e)-[:HAS_EVIDENCE]->(st)

    RETURN st.study_title
    """

    params = {
    "event_uid": depl_event_id,
    "study_no": study_obj.get('study_no'),
    "study_title": study_obj.get('study_title'),
    "content": study_obj.get('content'),
    "source": study_obj.get('source')
    }    

    try:
        db._driver_admin.execute_query(create_query,params)
        print(f"Studiul {study_obj.get('study_title')} a fost adaugat in KG cu numarul {study_obj.get('study_no')}")
            # return {"id": med_id, "name": medicament['Medicament'], "depletes_nutrient_name": medicament['Nutrients Depleted'][0]['Nutrient'] if medicament.get('Nutrients Depleted') else None}
    except Exception as e:
        print(f"Eroare la crearea studiului {study_obj.get('study_title')}: {e}")
    
